## Test Create CHCs

In this file we test the creation of CHCs from a given program.

First we import the necessary libraries and set up the Z3 solver.

In [80]:
from z3 import And, Ints, Int, Array, IntSort, ForAll, Implies, Or
import z3
import graphviz
z3.set_param(proof=True)
from sw.verify.stasm_to_chcs import create_chcs

# Sanity test
Now let's create a simple program and generate the CHCs for it.
This program is taken straight out of the class presentation.
It is a sanity test case for our implementation.
All it does is `stack[sp-2] + stack[sp-1] * 13`.  

In [81]:
program = [("PUSH", 13),
              ("POP", 2),
              ("ALU", "MUL"),
              ("POP", 2),
              ("ALU", "ADD"),
              ("POP", 1)]

In our case we are going to set 
`stack[0] = 1, stack[1] = 5`
and test that the output is `1 + 5 * 13`

In [82]:
stack = Array("stack", IntSort(), IntSort())
sp, r0, r1 = Ints('sp r0 r1')
chcs = create_chcs(pre_condition=And(stack[0] == 1, stack[1] == 5, sp == 2),
             input_vars=[],
             program=program,
             post_condition=stack[sp] == 1 + 5 * 13)
print(chcs.solve())

[U6 = [else ->
       Or(Exists(k!4,
                 And(k!4[1] == 5,
                     k!4[0] == 1,
                     Var(5) == k!4[0],
                     Var(4) == Var(2)[0],
                     Var(2) ==
                     Store(Store(Store(k!4, 2, 13),
                                 1,
                                 13*k!4[1]),
                           0,
                           13*k!4[1] + Var(5)),
                     Var(3) == 0)),
          Exists([k!2, k!1],
                 And(Exists(k!4,
                            And(k!4[1] == 5,
                                k!4[0] == 1,
                                Var(7) == k!4[0],
                                k!1 == 13*k!4[1],
                                Var(4) ==
                                Store(Store(Store(k!4,
                                        2,
                                        13),
                                        1,
                                        13*k!4[1]),
    

# Find max element in array
Now lets test the find max program from milestone 0
First, import some macros:

In [83]:
def READ_FROM_ARRAY():
    """
    Pops the top as index and then base addr and push the value at that index from the array at addr
    :return:
    """
    return [
        ("POP", 2),  # r0 = index, r1 = base addr
        ("ALU", "ADD"),  # push base addr + index
        ("POP", 1),   # r0 = base addr + index
        ("LOAD", 0),  # push value at base addr + index
    ]

Now writing the program:

In [84]:
program_find_max = [
    ("PUSH", 0),
    ("DUP", 2),
    *READ_FROM_ARRAY(),
    # Stack is now: [arr_addr, length, a[0]]
    # Now let's define max = a[0]
    ("DUP", 0),
    # Stack is now: [arr_addr, length, a[0], mx=a[0]]
    # Now after we saved a[0] on stack we can use this mem for i
    # lets set i = 1 to memory address &a[0]
    ("PUSH", 1),
    ("DUP", 4),
    ("POP", 2),
    ("STOR", 0),
    "CHECK_COND:",
    # First put i in r0 and n in r1
    ("DUP", 3),
    ("POP", 1),
    ("LOAD", 0),
    ("DUP", 3),
    ("POP", 2),
    # Check i < n
    ("ALU", "LT"),
    ("POP", 1),
    ("JNZ", "LOOP_BODY"),
    ("JMP", "END"),
    "LOOP_BODY:",
    # read i from mem and put on stack
    ("DUP", 3),
    ("POP", 1),
    ("LOAD", 0),
    # put base addr on stack
    ("DUP", 4),
    # Put a[i] on stack
    *READ_FROM_ARRAY(),
    # Put mx on stack
    ("DUP", 1),
    # now stack is [base_addr, n, a[0], mx, a[i], mx]
    ("POP", 2),
    ("ALU", "LT"),
    # now stack is [base_addr, n, a[0], mx, (result a[i] < mx)]
    ("POP", 1),
    ("JNZ", "INC_I"),
    ("JMP", "UPDATE"),
    "INC_I:",
    # Put i on stack
    ("PUSH", 0),
    ("DUP", 4),
    *READ_FROM_ARRAY(),
    ("PUSH", 1),
    ("POP", 2),
    ("ALU", "ADD"),
    ("DUP", 4),
    ("POP", 2),
    ("STOR", 0),
    ("JMP", "CHECK_COND"),
    "UPDATE:",
    ("POP", 1),  # remove old mx from stack
    # Push base addr
    ("DUP", 2),
    ("POP", 1),
    ("LOAD", 0),
    # Now i is on top of stack
    ("DUP", 3),
    *READ_FROM_ARRAY(),  # so now a[i] is where mx was on stack so mx = a[i]
    ("JMP", "INC_I"),
    "END:",
    # Restore a[0]:
    # Put a[0] on stack (we saved it before):
    ("DUP", 1),
    # Put base_addr on stack:
    ("DUP", 4),
    ("POP", 2),
    ("STOR", 0),
    ("POP", 2),
    ("POP", 1),
    ("POP", 1)
]

Now creating the verification (We are testing array of size 1: `[2]` and making sure the return value is 2 (`r1==2`)

In [85]:
stack = Array("stack", IntSort(), IntSort())
memory = Array("memory", IntSort(), IntSort())
sp, r0, r1 = Ints('sp r0 r1')

pre_condition = And(
    stack[0] == 0, stack[1] == 1, sp == 2,
    memory[0] == 2
)
chcs = create_chcs(pre_condition=pre_condition,
                   input_vars=[],
                   program=program_find_max,
                   post_condition=And(sp == 0,
                                      memory[0] == 2,
                                      r1 == 2))
print(chcs.solve())

[U31 = [else ->
        Or(Exists([k!7, k!6, k!4],
                  And(Not(k!7[-2 + k!6] <= 1),
                      Not(k!7[-1 + k!6] <= 0),
                      Not(k!7[-3 + k!6] >= 2),
                      Not(k!7[-2 + k!6] + -1*Var(3)[k!4] <=
                          0),
                      Not(k!6 >= 5),
                      Not(k!7[-2 + k!6] >= 3),
                      Not(Var(3)[0] <= 1),
                      Not(Var(3)[0] >= 3),
                      Var(7) ==
                      Var(3)[Store(Store(Store(Store(k!7,
                                        k!6,
                                        Var(3)[k!4]),
                                        1 + k!6,
                                        Store(k!7,
                                        k!6,
                                        Var(3)[k!4])[-2 +
                                        k!6]),
                                    k!6,
                                    Var(3)[Store(Store(k!7,
        

Now lets test that it finds model for `r1==9999` (the max is 2 and not 9999, so we should get a digraph for a counterexample)

In [86]:
stack = Array("stack", IntSort(), IntSort())
memory = Array("memory", IntSort(), IntSort())
sp, r0, r1 = Ints('sp r0 r1')
k = Int("k")

pre_condition = And(
    stack[0] == 0, stack[1] == 1, sp == 2,
    memory[0] == 2
)
chcs = create_chcs(pre_condition=pre_condition,
                   input_vars=[],
                   program=program_find_max,
                   post_condition=And(sp == 0,
                                      memory[0] == 2,
                                      r1 == 9999))
print(chcs.solve())

digraph {
	2 [label=False]
	310455 [label="query!127853(Store(Store(K(Int, 16), 1, 1), 0, 2),
             Store(Store(Store(Store(Store(Store(K(Int, 15),
                                        1,
                                        1),
                                        3,
                                        1),
                                     0,
                                     0),
                               4,
                               1),
                         2,
                         2),
                   5,
                   1),
             0,
             0,
             2)"]
	6358 [label="ForAll([A, B, C, D, E, F, G, H, I, J],
       Implies(And(U11(A, B, C, D, E),
                   G == Store(A, F[C], F[1 + C]),
                   H == F[-2 + C],
                   I == F[-4 + C],
                   J == -4 + C,
                   C >= 3,
                   C >= 4,
                   C >= 2,
                   C >= 1,
                 

Now lets test an array `[2,8,10,3]` of size 4 to make sure that we get SAT for `r1==10`

In [87]:
stack = Array("stack", IntSort(), IntSort())
memory = Array("memory", IntSort(), IntSort())
sp, r0, r1 = Ints('sp r0 r1')

pre_condition = And(
    stack[0] == 0, stack[1] == 4, sp == 2,
    memory[0] == 2, memory[1] == 8, memory[2] == 10, memory[3] == 3
)
# Test correct maximum
chcs_correct = create_chcs(pre_condition=pre_condition,
                           input_vars=[],
                           program=program_find_max,
                           post_condition=And(sp == 0,
                                              memory[0] == 2, memory[1] == 8,
                                              memory[2] == 10, memory[3] == 3,
                                              r1 == 10))
print(chcs_correct.solve())

[U31 = [else ->
        Or(Exists([k!7, k!6, k!5, k!4],
                  And(Or(Exists([k!9, k!8, k!7, k!6, k!5],
                                And(Exists([k!7, k!6, k!4],
                                        And(Not(k!7[-2 + k!6] >=
                                        3),
                                        Not(k!6 >= 5),
                                        Not(k!9[k!4] +
                                        -1*k!7[-2 + k!6] <=
                                        1),
                                        Not(k!7[-2 + k!6] +
                                        -1*k!7[-3 + k!6] >=
                                        -1),
                                        k!5 ==
                                        k!9[Store(Store(Store(Store(k!7,
                                        k!6,
                                        k!9[k!4]),
                                        1 + k!6,
                                        Store(k!7,
                     

# Test element exists in array program
Lets test the other program from milestone 

In [110]:
program_find_v = [
    ("PUSH", 0x0),  # int i = 0
    "CHECK_COND:",
    ("DUP", 0),  # Push i on stack
    ("DUP", 3),  # Push n on stack
    ("POP", 2),  # r0 = i, r1 = n
    ("ALU", "LT"),  # Compare i < n
    ("POP", 1),  # r0 = comparison result
    ("JNZ", "LOOP_BODY"),  # If i < n, jump to loop body
    ("JMP", "NOT_FOUND"),  # Otherwise, exit loop
    "INC_I:",  # Increment i
    ("PUSH", 1),  # Push constant 1
    ("POP", 2),  # r0 = i r1 = 1
    ("ALU", "ADD"),  # Push i + 1
    ("JMP", "CHECK_COND"),  # Jump to check condition
    "LOOP_BODY:",
    ("DUP", 0),  # Push i on stack
    ("DUP", 4),  # Push base_addr on stack
    *READ_FROM_ARRAY(),  # Push a[i]
    ("DUP", 2),
    ("POP", 2),
    ("ALU", "SUB"),
    ("POP", 1),  # r0 = a[i] - v
    ("JZ", "FOUND"),  # If a[i] == v, jump to found
    ("JMP", "INC_I"),  # Otherwise, check condition
    "FOUND:",  # Found the value
    ("POP", 2),  # Emptying the stack
    ("POP", 2),
    ("PUSH", 1),  # Push True
    ("PUSH", 1),  # Push True
    ("JMP", "END"),  # Jump to end
    "NOT_FOUND:",  # Not found
    ("POP", 2),  # Emptying the stack
    ("POP", 2),
    ("PUSH", 0),  # Push False
    ("PUSH", 0),  # Push False
    "END:",
    ("POP", 2)
]

Setting up the test

In [125]:
stack = Array("stack", IntSort(), IntSort())
memory = Array("memory", IntSort(), IntSort())
sp, r0, r1 = Ints('sp r0 r1')


k = Int("k")
pre_condition = And(
    stack[0] == 0, stack[1] == 4, stack[2] == 8,
    sp == 3,
    memory[0] == 2, memory[1] == 8, memory[2] == 1, memory[3] == 3,
    # Stack beyond sp is 0

)

# Test correct maximum
chcs_correct = create_chcs(pre_condition=pre_condition,
                           input_vars=[],
                           program=program_find_v,
                           post_condition=And(sp == 0,
                                              memory[0] == 2, memory[1] == 8, memory[2] == 1, memory[3] == 3,
                                              r1 == 1))
print(chcs_correct.solve())

digraph {
	2 [label=False]
	622098 [label="query!273759(Store(Store(Store(Store(K(Int, 15), 1, 8),
                               3,
                               3),
                         0,
                         2),
                   2,
                   1),
             Store(Store(Store(Store(Store(Store(K(Int, 16),
                                        1,
                                        0),
                                        3,
                                        0),
                                     0,
                                     0),
                               4,
                               0),
                         5,
                         8),
                   2,
                   8),
             0,
             0,
             0)"]
	930753 [label="ForAll([A, B, C, D, E, F, G, H, I],
       Implies(And(U5(A, B, C, D, E),
                   0 == B[-1 + C],
                   G == F[-5 + C],
                   H == F[-4 + C]